In [1]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gensim
import warnings
from gensim.models import FastText
import pickle

warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [2]:
dataset = pd.read_csv("News_dataset_preprocessed.csv")
articles = [w for w in dataset["text"]]
tokenized_words = [gensim.utils.simple_preprocess(str(w)) for w in articles]

In [3]:
# fastText model
fastText = FastText(vector_size = 300, window = 5, min_count = 1,
 sentences = tokenized_words, epochs = 10)

In [4]:
# 10 most similar words to "democrat" - fastText
print(fastText.wv.most_similar('democrat'))

[('democratno', 0.9510200619697571), ('mobsdemocrat', 0.9479070901870728), ('democratslou', 0.9471935629844666), ('democratsso', 0.9455419778823853), ('democratswe', 0.9427361488342285), ('democratnow', 0.9386740326881409), ('democratsof', 0.9374390244483948), ('democratsus', 0.9347637891769409), ('democratsit', 0.9273586869239807), ('democratskurtz', 0.9270105361938477)]


In [5]:
print("Similarity between the words 'democrat' " +
"and 'republican' - fastText : ",
fastText.wv.similarity('democrat', 'republican'))

Similarity between the words 'democrat' and 'republican' - fastText :  0.57389843


In [6]:
def display_closestwords_tsnescatterplot(model, word, size):
    
    arr = np.empty((0, size), dtype = 'f')
    word_labels = [word]

    close_words = model.similar_by_word(word)
    
    arr = np.append(arr, np.array([model[word]]), axis = 0)
    for wrd_score in close_words:
        wrd_vector = model[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis = 0)
        
        tsne = TSNE(n_components = 2, random_state = 0, perplexity = 1.9)
        np.set_printoptions(suppress = True)
        Y = tsne.fit_transform(arr)
    
    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    plt.scatter(x_coords, y_coords)
    
    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy = (x, y), xytext = (0, 0), textcoords = 'offset points')
    plt.xlim(x_coords.min() + 0.00005, x_coords.max() + 0.00005)
    plt.ylim(y_coords.min() + 0.00005, y_coords.max() + 0.00005)
    plt.title("Top 10 closest words to 'democrat' - fastText", pad = 20)
    plt.show()

In [7]:
display_closestwords_tsnescatterplot(fastText.wv, 'democrat', 300)

In [8]:
def getWordVectorFastText(doc):
    return [fastText.wv[word] for word in doc if word in fastText.wv.index_to_key]

In [9]:
word_vectors_fastText = []
for i in range(len(tokenized_words)):
    word_vectors_fastText.append(getWordVectorFastText(tokenized_words[i]))

In [10]:
# saving the vectors to disk
pickle_out = open("fasttext_vectors.pickle", "wb")
pickle.dump(word_vectors_fastText, pickle_out)
pickle_out.close()